In [1]:
%load_ext autoreload
%autoreload 2


In [6]:
# 標準ライブラリ
import os
from datetime import datetime, timedelta
from typing import List, Optional
from matplotlib.figure import Figure
from dataclasses import dataclass

# サードパーティライブラリ
import polars as pl
import pandas as pd

#import pandas as pd
#from pandas import Timedelta
import matplotlib.pyplot as plt
import japanize_matplotlib
from matplotlib.ticker import ScalarFormatter
from IPython.display import Markdown, display

# プロジェクト内モジュール
import gamedata as g



In [3]:
df_info = g.load_hard_info()
df_info

id,launch_date,maker_name,full_name
"""PS5""",2020-11-12 00:00:00,"""SONY""","""PlayStation5"""
"""XSX""",2020-11-10 00:00:00,"""Microsoft""","""Xbox Series X|S"""
"""NSW""",2017-03-03 00:00:00,"""Nintendo""","""Nintendo Switch"""
"""NS2""",2025-06-05 00:00:00,"""Nintendo""","""Nintendo Switch2"""
"""XBOne""",2014-09-04 00:00:00,"""Microsoft""","""XboxOne"""
"""PS4""",2014-02-22 00:00:00,"""SONY""","""PlayStation4"""
"""WiiU""",2012-12-08 00:00:00,"""Nintendo""","""WiiU"""
"""Vita""",2011-12-17 00:00:00,"""SONY""","""PlayStation Vita"""
"""3DS""",2011-02-26 00:00:00,"""Nintendo""","""Nintendo 3DS"""
"""Wii""",2006-12-02 00:00:00,"""Nintendo""","""Wii"""


In [4]:
df = g.load_hard_sales()
current_df = g.date_filter(df, begin=datetime(2026, 1, 10))

current_style = g.chart_units_by_date_hw(current_df)
current_style


In [10]:
DB_PATH = '/Users/hide/Documents/sqlite3/gamehard.db'
import sqlite3


In [8]:
def load_hard_event_pd() -> pd.DataFrame:
    """
    sqlite3を使用してデータベースからハードウェアイベントデータを読み込む関数。
    日付関係のカラムをdatetime64[ns]型に変換して返す。
    
    Returns:
        pd.DataFrame: ハードウェアイベントデータのDataFrame。
                      日付カラム（event_date）はdatetime64[ns]型に変換済み。
    """
    # SQLite3データベースに接続
    conn = sqlite3.connect(DB_PATH)
    # SQLクエリを実行してデータをDataFrameに読み込む
    query = "SELECT event_date,hw,event_name,event_type,priority FROM gamehard_event ORDER BY event_date;"
    df = pd.read_sql_query(query, conn)
    
    # 接続を閉じる
    conn.close()

    # 日付をdatetime64[ns]型に変換
    df['event_date'] = pd.to_datetime(df['event_date'])
    
    # report_dateカラムを作成する。event_dateが日曜日ならそのまま、そうでなければ直近の日曜日を設定
    df['report_date'] = df['event_date'].apply(
        lambda d: d if d.weekday() == 6 else d + pd.offsets.Week(weekday=6)
    )
    df.reset_index(inplace=True)
    return df


In [11]:
pddf_event = load_hard_event_pd()
pddf_event

,index,event_date,hw,event_name,event_type,priority,report_date
0,0,1999-02-11,PS,FF8,soft,0.2,1999-02-14
1,1,1999-11-21,GB,ポケモン金銀,soft,2.0,1999-11-21
2,2,1999-12-11,PS,GT2,soft,0.3,1999-12-12
3,3,1999-12-29,DC,シェンムー,soft,0.4,2000-01-02
4,4,2000-07-07,PS,FF9,soft,1.5,2000-07-09
...,...,...,...,...,...,...,...
142,142,2026-02-27,NS2,バイオ789,soft,1.5,2026-03-01
143,143,2026-02-27,PS5,バイオ9,soft,1.5,2026-03-01
144,144,2026-03-05,NS2,ぽこあポケ,soft,1.0,2026-03-08
145,145,2026-03-13,NS2,MHストーリーズ3,soft,3.0,2026-03-15


In [16]:
event_df = g.load_hard_event()
event_df

event_date,hw,event_name,event_type,priority,report_date
1999-02-11 00:00:00,"""PS""","""FF8""","""soft""",0.20,1999-02-14 00:00:00
1999-11-21 00:00:00,"""GB""","""ポケモン金銀""","""soft""",2.00,1999-11-21 00:00:00
1999-12-11 00:00:00,"""PS""","""GT2""","""soft""",0.30,1999-12-12 00:00:00
1999-12-29 00:00:00,"""DC""","""シェンムー""","""soft""",0.40,2000-01-02 00:00:00
2000-07-07 00:00:00,"""PS""","""FF9""","""soft""",1.50,2000-07-09 00:00:00
2000-08-26 00:00:00,"""PS""","""DQ7""","""soft""",1.50,2000-08-27 00:00:00
2001-01-31 00:00:00,"""DC""","""DC撤退発表""","""hard""",0.50,2001-02-04 00:00:00
2001-04-28 00:00:00,"""PS2""","""GT3""","""soft""",1.00,2001-04-29 00:00:00
2001-07-19 00:00:00,"""PS2""","""FFX""","""soft""",0.20,2001-07-22 00:00:00
2001-09-06 00:00:00,"""DC""","""シェンムー2""","""soft""",1.80,2001-09-09 00:00:00


In [18]:
masked_event = g.mask_event(event_df, g.EVENT_MASK_MIDDLE)
masked_event

event_date,hw,event_name,event_type,priority,report_date
1999-02-11 00:00:00,"""PS""","""FF8""","""soft""",0.20,1999-02-14 00:00:00
1999-12-11 00:00:00,"""PS""","""GT2""","""soft""",0.30,1999-12-12 00:00:00
1999-12-29 00:00:00,"""DC""","""シェンムー""","""soft""",0.40,2000-01-02 00:00:00
2000-07-07 00:00:00,"""PS""","""FF9""","""soft""",1.50,2000-07-09 00:00:00
2000-08-26 00:00:00,"""PS""","""DQ7""","""soft""",1.50,2000-08-27 00:00:00
2001-01-31 00:00:00,"""DC""","""DC撤退発表""","""hard""",0.50,2001-02-04 00:00:00
2001-04-28 00:00:00,"""PS2""","""GT3""","""soft""",1.00,2001-04-29 00:00:00
2001-07-19 00:00:00,"""PS2""","""FFX""","""soft""",0.20,2001-07-22 00:00:00
2001-12-31 00:00:00,"""DC""","""DC販売終了""","""hard""",1.50,2002-01-06 00:00:00
2002-07-19 00:00:00,"""GC""","""マリオサンシャイン""","""soft""",0.30,2002-07-21 00:00:00
